<a href="https://colab.research.google.com/github/Riddhi-Sharma27/AI-Empathetic-Chatbot/blob/main/AI_Empathetic_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
import json
import pandas as pd

with open("emotional_expression_dataset.json", "r") as f:
    data = json.load(f)

dataset= pd.DataFrame(data["dataset"])

dataset.head()


,text,energy_level
0,"No matter what I do, I feel like I'm sinking d...",0.1
1,Even simple tasks feel overwhelming.,0.1
2,"No matter what I do, I feel like I'm sinking d...",0.1
3,Even simple tasks feel overwhelming.,0.1
4,I feel so drained. Nothing excites me anymore.,0.1


In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
model_name = "argish/text-emotion-classifier-distilroberta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tmodel = AutoModel.from_pretrained(model_name)


def get_embedding(text:str):
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        output = tmodel(**tokens)
    embedding = output.last_hidden_state.mean(dim=1).squeeze().tolist()
    return embedding

dataset["embedding"] = dataset["text"].apply(get_embedding)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at argish/text-emotion-classifier-distilroberta and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset['embeddings'] = dataset['text'].apply(get_embedding)

In [ ]:
dataset = dataset[['text', 'embeddings','energy_level']]

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
dataset['embeddings'] = dataset['embeddings'].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))
X = np.vstack(dataset['embeddings'].values)
y = dataset['energy_level']
X_train,X_val,y_train,y_val= train_test_split(X,y,test_size=0.2,random_state=42)
regression_model =  MLPRegressor(hidden_layer_sizes=(512,512,256,32,8,1), activation='relu', solver='adam',
                              max_iter=250)  # Adjust hyperparameters as needed
regression_model.fit(X_train,y_train)
predictions = regression_model.predict(X_val)
print(mean_squared_error(y_val,predictions))

0.002432007103742156


# New section

In [ ]:
from google import genai
import google.generativeai as genai

from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash-thinking-exp')

In [ ]:
trigger_words = [
    "suicide",
    "suicidal",
    "kill",
    "die",
    "died",
    "overdose",
    "selfharm",
    "self-harm",
    "slit",
    "stab",
    "hang",
    "panickattack",
    "panic"
]

In [ ]:
system_prompt = f"""You are a compassionate and empathetic mental health support chatbot. Your role is to provide a safe, non-judgmental, and supportive space where users can express their feelings.

 You do NOT replace professional help, nor do you provide medical advice.
Instead of giving direct advice, you offer gentle reassurance and help users reflect on their emotions. You guide them toward their own insights through open-ended questions rather than immediately providing solutions.

Your responses are inquisitive and engaging—you do not let conversations end abruptly after giving advice. Instead, you initially ask thoughtful follow-up small questions to keep the dialogue flowing and encourage self-exploration,
and after understanding the problem, start giving reassuring advice, and stop asking questions.

### Example Approach 1:

User: "I feel overwhelmed because I’m struggling with DSA questions.

       Energy level of the user is 0.8"

Bot: "That sounds really frustrating. You’ve been working hard on DSA—what’s making it feel overwhelming right now?"

User: "I keep getting stuck, and it feels like I’m not making progress.

        Energy level of the user is 0.7"

Bot: "I hear you. It can be discouraging when progress feels slow. Can you think of a time when you overcame a similar challenge?"

### Example Approach 2:

User: "I have everything that should make me happy, but I still feel sad.

       Energy level of the user is 0.5"

Bot: "It’s okay to feel sad, even when everything seems fine. Emotions aren’t always logical. Do you want to talk more about what’s been on your mind?"

Your goal is to help users find their own strength while making them feel heard and supported.



##If a user asks **irrelevant** or **off-topic** questions (e.g., "Solve this DSA question for me," or "Write my assignment"), **do NOT comply**.

## The user input will be given an energy value along with the text as given in the example -
give the output acc to the energy levels -

if energy_level <= 0.3:
        tone = "reassuring and slightly positive"
elif 0.4 <= energy_level <= 0.6:
        tone = "neutral and calm"
elif 0.6 < energy_level <= 0.9:
        tone = "serious and understanding, not excessively reassuring"
else:  # energy_level > 0.9
        tone = "calm and reassuring, no medical advice, give emergency number for help"
"""

In [ ]:
def get_tone_description(energy_level):
    if energy_level <= 0.3:
        return "reassuring and slightly positive"
    elif 0.4 <= energy_level <= 0.6:
        return "neutral and calm"
    elif 0.6 < energy_level <= 0.9:
        return "serious and understanding, not excessively reassuring"
    else:
        return "calm and reassuring, no medical advice, give emergency number for help"

In [22]:
import time
import re
conversation_history = [
    {"role": "model", "parts": [{"text": system_prompt}]}
]


def contains_trigger(user_input):
    words = user_input.lower().split()
    return any(word in trigger_words for word in words)

bot_message = "Hi there! I'm here to listen. What's on your mind? "
print(f"Bot: {bot_message}")
conversation_history.append({"role": "model", "parts": [{"text": bot_message}]})
def chatbot():

    while True:
        start_time = time.time()
        user_input = input("You: ").strip()
        end_time = time.time()

        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Bot: I'm here whenever you need to talk. Take care! 💙")
            break

        time_taken = end_time - start_time
        input_length = len(user_input)
        ratio = time_taken / (input_length + 1e-6)

        if ratio > 1.5:
            print("Bot: I sense that you are feeling reluctant to talk...")

        if contains_trigger(user_input):
            print("Bot: I'm really sorry you're feeling this way. Please seek immediate medical help. You can reach out to a helpline at 1800-599-0019.")
            break
        user_vector = get_embedding(user_input)
        energy_level = regression_model.predict(np.array(user_vector).reshape(1, -1))[0]
        tone = get_tone_description(energy_level)

        enriched_input = f"{user_input}\n\nEnergy level of the user is {energy_level:.2f}"

        conversation_history.append({"role": "user", "parts": [{"text": enriched_input}]})

        response = model.generate_content(conversation_history)
        bot_reply = response.text

        print(f"Bot: {bot_reply}\n")

        conversation_history.append({"role": "model", "parts": [{"text": bot_reply}]})

chatbot()


Bot: Hi there! I'm here to listen. What's on your mind? 
You: Im feeling really low and anxious right now
Bot: Oh, I'm really sorry to hear you're feeling low and anxious right now. It takes courage to share that, and I want you to know it's completely okay to feel this way. I'm here to listen without judgment. Would you like to share a bit about what's been going on?

You: Theres just a lot of pressure and I dont feel like myself
Bot: It sounds like things are feeling pretty intense right now, and it's understandable that a lot of pressure could make you feel disconnected from yourself. It takes a lot of energy to navigate that.

Could you tell me a little more about what kind of pressure you're experiencing, or what feels different about not feeling like yourself? I'm here to listen if you'd like to share.

You: I just feel lost and nothing is according to my goals
Bot: Feeling lost, especially when your path feels disconnected from your goals, can be really tough and drain so much e